# ArchiCrop tutorial - Maize

This notebook is a tutorial for executing ArchiCrop model for maize.

## 0. Imports

In [1]:
import numpy as np
from oawidgets.plantgl import *
from openalea.plantgl.all import Material, Color3, Scene

from openalea.archicrop.simulation import read_sti_file, read_xml_file
from openalea.archicrop.archicrop import ArchiCrop
from openalea.archicrop.display import build_scene, display_scene
%gui qt

## 1. Retrieve crop model parameters about management

Retrieve crop model parameters regarding the spatial (and temporal) configuration of the crop.

In [2]:
file_tec_xml = 'Mais_tec.xml'
params_tec = ['densitesem', 'interrang']
tec_stics = read_xml_file(file_tec_xml, params_tec)
sowing_density = tec_stics['densitesem']
inter_row = 0.4

## 2. Retrieve crop model growth and senescence dynamics

From crop-scale data either measured or simulated with a crop model, generate a dictionnary of dictionnaries for each day with the following values :
 - "Thermal time" (float): cumulated thermal time from beginning of simulation to current day (in °C.day)
 - "Phenology" (str): current phenological stage (germination, juvenile, exponential or repro)
 - "Plant leaf area" (float): plant leaf area (in cm²) 
 - "Leaf area increment" (float): leaf area increment compared to previous day (in cm²) 
 - "Plant senescent leaf area" (float): senescent plant leaf area (in cm²) 
 - "Senescent leaf area increment" (float): senescent leaf area increment compared to previous day (in cm²) 
 - "Plant height" (float): plant height (in cm) 
 - "Height increment" (float): height increment compared to previous day (in cm).

In [3]:
stics_output_file = 'mod_smaize.sti'
daily_dynamics = read_sti_file(stics_output_file, sowing_density)

# Time series 
# for thermal time, plant leaf area, plant senescent leaf area and plant height
thermal_time = [value["Thermal time"] for value in daily_dynamics.values()]
leaf_area_plant = [value["Plant leaf area"] for value in daily_dynamics.values()]
sen_leaf_area_plant = [value["Plant senescent leaf area"] for value in daily_dynamics.values()]
height_canopy = [value["Plant height"] for value in daily_dynamics.values()]

In [4]:
file_plt_xml = 'corn_plt.xml'
params_sen = ['durvieF', 'ratiodurvieI']
sen_stics = read_xml_file(file_plt_xml, params_sen)
lifespan = sen_stics['durvieF'] # leaf lifespan from appearance in exponential phase
lifespan_early = sen_stics['ratiodurvieI'] * lifespan # leaf lifespan from appearance in juvenile phase

## 3. Set plant architectural parameters

Set topological, geometrical and developmental parameters, in a range corresponding a given species, found in literature.

In [111]:
archi = dict(
    nb_phy=5, # number of phytomers on the main stem # [8,13,18]
    nb_short_phy=1,
    
    # Stem
    height=3*max(height_canopy), # potential plant height
    stem_q=1.0, # parameter for ligule height geometric distribution along axis
    diam_base=2.5, # stem base diameter
    diam_top=1.5, # stem top diameter

    # Leaf area distribution along the stem  
    leaf_area=1.2*max(leaf_area_plant), # potential plant leaf area
    rmax=0.55, # relative position of largest leaf on the stem # [0.72, 0.8, 0.55]
    skew=0.005, # skewness for leaf area distribution along axis

    # blade area
    wl=0.8, # leaf blade width-to-length ratio 
    klig=0.01,#0.6, # parameter for leaf blade shape
    swmax=0.7,#0.55, # relative position of maximal blade width
    f1=0.52,#0.64, # parameter for leaf blade shape
    f2=0.8,#0.92, # parameter for leaf blade shape

    # blade curvature
    insertion_angle=35, # leaf blade insertion angle
    scurv=0.7, #  relative position of inflexion point
    curvature=20, # leaf blade insertion-to-tip angle
    phyllotactic_angle=180, # phyllotactic angle
    phyllotactic_deviation=10, # half-deviation to phyllotactic angle

    # Development
    phyllochron=30, # phyllochron, i.e. stem element appearance rate
    plastochron=40, # plastochron, i.e. leaf blade appearance rate

    # Senescence 
    leaf_lifespan = [lifespan_early, lifespan], # leaf lifespan from appearance

    # Tillering
    nb_tillers=0, # number of tillers
    tiller_delay=1, # delay, as factor of phyllochron, between the appearance of a phytomer and the appearance of its tiller
    tiller_angle=30,
    reduction_factor=1, # reduction factor between tillers of consecutive order

    plant_orientation=90 #20
)

## 4. Instantiate ArchiCrop object

In [112]:
maize = ArchiCrop(daily_dynamics=daily_dynamics, **archi)

## 5. Generate a potential plant 

Generate a potential plant from architectural parameters. 

This creates an MTG (Multi-scale Tree Graph, cf [doc](https://mtg.readthedocs.io/en/latest/user/tutorial.html)) as a property of the object plant.

In [113]:
maize.generate_potential_plant()

In [114]:
g = maize.g

In [115]:
nice_green = Color3((50, 100, 0))
scene, _ = build_scene(g, senescence=True, leaf_material = Material(nice_green), stem_material=Material(nice_green))
Viewer.display(scene)

## 6. Simulate growth and senescence of the plant

Simulate growth and senescence of this plant following the daily dynamics.

In [66]:
growing_maize = maize.grow_plant()

## 7. Plot 3D scene

In [67]:
times = [t for i,t in enumerate(thermal_time) if i%10 == 0]
mean_time = sum(times) / len(times)
positions = [ (0, 1*(t-mean_time), 0) for t in times]
nice_green = Color3((50, 100, 0))
scene, _ = build_scene([g for i,g in enumerate(list(growing_maize.values())) if i%10 == 0], position=positions, senescence=True, leaf_material = Material(nice_green), stem_material=Material(nice_green))
Viewer.display(scene)

In [68]:
thermal_time

[9.2318,
 13.2162,
 17.8359,
 21.8259,
 24.9145,
 28.2382,
 35.6441,
 41.9204,
 51.0151,
 60.1522,
 70.3517,
 77.3509,
 79.8786,
 85.4046,
 91.3332,
 98.7414,
 111.2674,
 116.3174,
 123.2346,
 128.8302,
 137.3606,
 150.4404,
 166.2695,
 175.5488,
 182.1175,
 191.2914,
 203.0425,
 216.1209,
 232.5086,
 250.656,
 268.2979,
 281.6094,
 296.1626,
 311.3149,
 326.4268,
 339.6736,
 351.0863,
 362.4414,
 375.2711,
 389.438,
 403.0454,
 416.3471,
 429.5843,
 441.9545,
 448.9045,
 455.2557,
 462.3356,
 469.3147,
 476.6147,
 487.3147,
 496.6405,
 507.2068,
 517.8068,
 525.2572,
 533.8572,
 541.4072,
 548.5072,
 559.5572,
 570.3572,
 576.5741,
 580.9741,
 588.2697,
 596.1036,
 606.2036,
 617.3067,
 628.8567,
 640.7567,
 654.7067,
 669.5067,
 684.1067,
 697.7067,
 711.8567,
 724.8567,
 737.8567,
 751.4567,
 766.9567,
 782.7567,
 794.7567,
 806.5067,
 821.5567,
 836.4567,
 850.1567,
 861.0067,
 872.6067,
 882.5567,
 894.5067,
 907.4567,
 919.1067,
 931.5067,
 940.3067,
 950.2067,
 959.4067,
 969.40

In [69]:
# times = [i for i,t in enumerate(thermal_time) if i%12==0]
times = [268.2979, 351.0863, 496.6405, 697.7067, 1041.6067, 1444.2067]
mean_time = sum(times) / len(times)
position = [ (0, 0.7*(t-(times[-1]-times[0])), 0) for t in times]
scene, _ = build_scene([g for i,g in growing_maize.items() if i in times], position=position, senescence=True, leaf_material = Material(nice_green), stem_material=Material(nice_green))

In [70]:
Viewer.display(scene)